# 라이브러리 불러오기

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen

In [3]:
import os
os.chdir("C:/Users/PC/Jupyter_file/강서구_빅데이터공모전/data")

# 데이터 불러오기

In [3]:
ansim_df = pd.read_csv('C:/Users/PC/Jupyter_file/강서구_빅데이터공모전/data/서울시 강서구 안심이 CCTV 연계 현황.csv',encoding='cp949')
crosswalk_df = pd.read_csv('C:/Users/PC/Jupyter_file/강서구_빅데이터공모전/data/서울시 대로변 횡단보도 위치정보.csv',encoding='cp949')
bus_stop_df =pd.read_csv('C:/Users/PC/Jupyter_file/강서구_빅데이터공모전/data/서울시버스정류소위치정보(20240305).csv',encoding='cp949')

# 행정동기준으로 데이터 분리

## 횡단보도 데이터

In [26]:
crosswalk_df.head()

,노드링크 유형,노드 WKT,노드 ID,노드 유형 코드,링크 WKT,링크 ID,링크 유형 코드,시작노드 ID,종료노드 ID,링크 길이,시군구코드,시군구명,읍면동코드,읍면동명
16629,NODE,POINT(126.82926578437203 37.55519774185072),72452,0.0,NaN,0,NaN,NaN,NaN,NaN,1150000000,강서구,1150010600,내발산동
16630,NODE,POINT(126.8160581981105 37.54927076447105),149423,0.0,NaN,0,NaN,NaN,NaN,NaN,1150000000,강서구,1150010700,외발산동
16631,NODE,POINT(126.81641591878092 37.55122832217836),149424,0.0,NaN,0,NaN,NaN,NaN,NaN,1150000000,강서구,1150010700,외발산동
16632,NODE,POINT(126.8169188706517 37.55125885401416),149425,0.0,NaN,0,NaN,NaN,NaN,NaN,1150000000,강서구,1150010700,외발산동
16633,LINK,NaN,0,NaN,LINESTRING(126.81171373760999 37.5771256174733...,62545,1000.0,162389.0,162390.0,13.188,1150000000,강서구,1150010900,방화동


### 강서구 데이터만 추출

In [6]:
crosswalk_df = crosswalk_df.loc[crosswalk_df['시군구명']=='강서구']

In [11]:
crosswalk_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1649 entries, 16629 to 18277
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   노드링크 유형   1649 non-null   object 
 1   노드 WKT    998 non-null    object 
 2   노드 ID     1649 non-null   int64  
 3   노드 유형 코드  998 non-null    float64
 4   링크 WKT    651 non-null    object 
 5   링크 ID     1649 non-null   int64  
 6   링크 유형 코드  651 non-null    float64
 7   시작노드 ID   651 non-null    float64
 8   종료노드 ID   651 non-null    float64
 9   링크 길이     651 non-null    float64
 10  시군구코드     1649 non-null   int64  
 11  시군구명      1649 non-null   object 
 12  읍면동코드     1649 non-null   int64  
 13  읍면동명      1649 non-null   object 
dtypes: float64(5), int64(4), object(5)
memory usage: 193.2+ KB


어차피 링크에 횡단보도 두 지점의 좌표가 있어서 링크만 남기고 나머지 없앰

In [32]:
crosswalk_df = crosswalk_df.dropna(subset=['링크 WKT'])

In [34]:
# 횡단보도 경도 및 위도 열 추가
crosswalk_df['횡단보도 경도'] = crosswalk_df['링크 WKT'].apply(lambda x: float(x.split('(')[1].split()[0]))
crosswalk_df['횡단보도 위도'] = crosswalk_df['링크 WKT'].apply(lambda x: float(x.split()[1].split(',')[0]))

C:\Users\PC\AppData\Local\Temp\ipykernel_19096\4172569583.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crosswalk_df['횡단보도 경도'] = crosswalk_df['링크 WKT'].apply(lambda x: float(x.split('(')[1].split()[0]))
C:\Users\PC\AppData\Local\Temp\ipykernel_19096\4172569583.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crosswalk_df['횡단보도 위도'] = crosswalk_df['링크 WKT'].apply(lambda x: float(x.split()[1].split(',')[0]))


In [35]:
crosswalk_df.head()

,노드링크 유형,노드 WKT,노드 ID,노드 유형 코드,링크 WKT,링크 ID,링크 유형 코드,시작노드 ID,종료노드 ID,링크 길이,시군구코드,시군구명,읍면동코드,읍면동명,횡단보도 경도,횡단보도 위도
16633,LINK,NaN,0,NaN,LINESTRING(126.81171373760999 37.5771256174733...,62545,1000.0,162389.0,162390.0,13.188,1150000000,강서구,1150010900,방화동,126.811714,37.577126
16634,LINK,NaN,0,NaN,LINESTRING(126.81632360709798 37.5652725812043...,2450,1000.0,85316.0,85321.0,41.929,1150000000,강서구,1150010500,마곡동,126.816324,37.565273
16635,LINK,NaN,0,NaN,LINESTRING(126.8254384095112 37.55821758307206...,85142,1000.0,119246.0,119272.0,7.383,1150000000,강서구,1150010500,마곡동,126.825438,37.558218
16636,LINK,NaN,0,NaN,LINESTRING(126.81080518088379 37.5793669163331...,5820,1000.0,116808.0,116806.0,11.728,1150000000,강서구,1150010900,방화동,126.810805,37.579367
16637,LINK,NaN,0,NaN,LINESTRING(126.83953169546966 37.5281855257082...,5026,1000.0,174676.0,174679.0,11.478,1150000000,강서구,1150010300,화곡동,126.839532,37.528186


In [36]:
crosswalk_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 651 entries, 16633 to 17960
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   노드링크 유형   651 non-null    object 
 1   노드 WKT    0 non-null      object 
 2   노드 ID     651 non-null    int64  
 3   노드 유형 코드  0 non-null      float64
 4   링크 WKT    651 non-null    object 
 5   링크 ID     651 non-null    int64  
 6   링크 유형 코드  651 non-null    float64
 7   시작노드 ID   651 non-null    float64
 8   종료노드 ID   651 non-null    float64
 9   링크 길이     651 non-null    float64
 10  시군구코드     651 non-null    int64  
 11  시군구명      651 non-null    object 
 12  읍면동코드     651 non-null    int64  
 13  읍면동명      651 non-null    object 
 14  횡단보도 경도   651 non-null    float64
 15  횡단보도 위도   651 non-null    float64
dtypes: float64(7), int64(4), object(5)
memory usage: 86.5+ KB


### 역지오코딩으로 횡단보도 위치별 행정동 가져오기

In [37]:
## 카카오 API를 활용한 역 지오코딩(행정동 기준)

import pandas as pd
import json
import requests
from datetime import datetime

APP_KEY = "a5c961416d75a72774f0451df047b671"
URL = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"

def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': f'KakaoAK {APP_KEY}'}
    resp = requests.get(url, headers=headers)
    print('%s : success for request [%s]' % (datetime.now(), url))

    return resp.text

# 빈 리스트 초기화
region_type_list = []
code_list = []
address_name_list = []
region_1depth_name_list = []
region_2depth_name_list = []
region_3depth_name_list = []
region_4depth_name_list = []
x_list = []
y_list = []

# 데이터프레임에 있는 각 좌표에 대해 요청 실행
for index, row in crosswalk_df.iterrows():
    lat = row['횡단보도 위도']
    lng = row['횡단보도 경도']
    # 행정동 정보를 가져오기 위해 region_type 매개변수 추가하여 URL 업데이트
    response_text = json_request(f"{URL}?x={lng}&y={lat}&input_coord=WGS84&output_coord=TM&region_type=H")
    response_json = json.loads(response_text)

    if 'documents' in response_json:
        # [0]은 법정동 데이터프레임
        # [1]은 행정동 데이터프레임 만들 수 있음
        document = response_json['documents'][1]

        region_type_list.append(document.get('region_type', ''))
        code_list.append(document.get('code', ''))
        address_name_list.append(document.get('address_name', ''))
        region_1depth_name_list.append(document.get('region_1depth_name', ''))
        region_2depth_name_list.append(document.get('region_2depth_name', ''))
        region_3depth_name_list.append(document.get('region_3depth_name', ''))
        region_4depth_name_list.append(document.get('region_4depth_name', ''))
        x_list.append(document.get('x', ''))
        y_list.append(document.get('y', ''))

# 결과 데이터프레임 생성
cw_result_df = pd.DataFrame({
    'region_type': region_type_list,
    'code': code_list,
    'address_name': address_name_list,
    'region_1depth_name': region_1depth_name_list,
    'region_2depth_name': region_2depth_name_list,
    'region_3depth_name': region_3depth_name_list,
    'region_4depth_name': region_4depth_name_list,
    'x': x_list,
    'y': y_list
})


2024-03-26 03:51:34.979099 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.81171373760999&y=37.577125617473385&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:51:35.416283 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.81632360709798&y=37.5652725812043&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:51:35.847546 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8254384095112&y=37.55821758307206&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:51:36.226616 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.81080518088379&y=37.57936691633317&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:51:36.731404 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83953169546966&y=37.5281855257082&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 0

2024-03-26 03:51:55.454435 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8481728755811&y=37.56366222819339&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:51:55.930307 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.82616324005753&y=37.55556853390644&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:51:56.536173 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.82468903176922&y=37.56682318972357&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:51:57.099620 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.84344124495094&y=37.543621853619186&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:51:57.539345 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.84574617376789&y=37.53363160268837&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26

2024-03-26 03:52:15.664276 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.79855113993312&y=37.58210515741631&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:52:16.194709 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.81270431672486&y=37.56775225368634&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:52:16.762366 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83791308453328&y=37.558341611253134&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:52:17.273221 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.85185305769834&y=37.55468343875738&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:52:17.766392 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83310478287258&y=37.55731197192662&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-2

2024-03-26 03:52:37.744234 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.82720300286643&y=37.55095409078206&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:52:38.360071 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.82347475988833&y=37.55831307034267&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:52:38.823719 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83418775404894&y=37.5638824195135&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:52:39.375200 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8481728755811&y=37.56366222819339&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:52:40.009902 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8243750771316&y=37.56339450894014&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03

2024-03-26 03:53:00.675491 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.84437450750583&y=37.54683519332532&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:53:01.188454 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.85454308079115&y=37.55497971544088&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:53:01.838085 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.820211478217&y=37.55869679525197&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:53:02.463783 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83419970003206&y=37.56368104580007&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:53:02.902545 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.82995741148918&y=37.560672462214924&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 

...

2024-03-26 03:55:24.989283 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.85483709199832&y=37.56087763578645&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:55:25.551209 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8246032425746&y=37.549046197855446&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:55:26.076845 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8328655616944&y=37.55741819045961&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:55:26.564161 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.82436102010438&y=37.56448382732503&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:55:27.133434 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.86090465344387&y=37.552641015229895&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26

2024-03-26 03:55:46.685000 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83121222992371&y=37.553131707516485&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:55:47.192091 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.85138928741186&y=37.56674798599159&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:55:47.824979 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.84139656071092&y=37.5401475604705&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:55:48.303838 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.84690496379773&y=37.56129816128497&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:55:48.857505 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.84511691939859&y=37.54642423310311&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26

2024-03-26 03:56:07.526140 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.84006912544302&y=37.560563505634875&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:56:08.090020 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.85228132417346&y=37.556162367037324&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:56:08.599383 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.85016322038716&y=37.55675150282517&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:56:09.173332 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.81164675600402&y=37.57346616217297&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:56:09.716451 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.84048615122941&y=37.56496780907596&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-

2024-03-26 03:56:29.537535 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83556750348285&y=37.560149912377625&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:56:29.952992 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83873991331518&y=37.56058922141269&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:56:30.370194 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83272385648088&y=37.55516937450024&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:56:30.848215 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.85363454068879&y=37.565354788845916&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:56:31.302275 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83614111568976&y=37.548127194971656&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03

2024-03-26 03:56:48.301440 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83026265854372&y=37.56044404850234&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:56:48.822308 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.80504724188249&y=37.56751014424153&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:56:49.261980 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83923210813307&y=37.5607817953924&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:56:49.722560 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.81129091661785&y=37.564328711866025&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 03:56:50.170657 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.83049046356962&y=37.56140555808763&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26

### 결과 확인

In [41]:
cw_result_df.head()

,region_type,code,address_name,region_1depth_name,region_2depth_name,region_3depth_name,region_4depth_name,x,y
0,H,1150064100,서울특별시 강서구 방화3동,서울특별시,강서구,방화3동,,183460.659071,452954.975726
1,H,1150063000,서울특별시 강서구 방화1동,서울특별시,강서구,방화1동,,183807.043961,451835.760924
2,H,1150061100,서울특별시 강서구 발산1동,서울특별시,강서구,발산1동,,185197.027592,450105.324369
3,H,1150064100,서울특별시 강서구 방화3동,서울특별시,강서구,방화3동,,183460.659071,452954.975726
4,H,1150054000,서울특별시 강서구 화곡1동,서울특별시,강서구,화곡1동,,185929.393158,447584.491911


이상한 목동1동 삭제

In [42]:
cw_result_df = cw_result_df.groupby('region_3depth_name').size().reset_index(name='횡단보도 수')
cw_result_df.drop([7],axis=1,inplace=True)

In [50]:
cw_result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 0 to 20
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   region_3depth_name  20 non-null     object
 1   횡단보도 수              20 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 480.0+ bytes


데이터 저장

In [53]:
cw_result_df.to_csv('cw_result_df.csv', index=False)

In [40]:
bus_stop_df.head()

,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,정류소타입
0,100000001,1001,종로2가사거리,126.987752,37.569806,중앙차로
1,100000002,1002,창경궁.서울대학교병원,126.996521,37.579433,중앙차로
2,100000003,1003,명륜3가.성대입구,126.998251,37.582580,중앙차로
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,중앙차로
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,중앙차로


In [39]:
bus_stop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11261 entries, 0 to 11260
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   NODE_ID  11261 non-null  int64  
 1   ARS_ID   11261 non-null  int64  
 2   정류소명     11261 non-null  object 
 3   X좌표      11261 non-null  float64
 4   Y좌표      11261 non-null  float64
 5   정류소타입    11261 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 528.0+ KB


In [54]:
## 카카오 API를 활용한 역 지오코딩(행정동 기준)

# 빈 리스트 초기화
region_type_list = []
code_list = []
address_name_list = []
region_1depth_name_list = []
region_2depth_name_list = []
region_3depth_name_list = []
region_4depth_name_list = []
x_list = []
y_list = []

# 데이터프레임에 있는 각 좌표에 대해 요청 실행
for index, row in bus_stop_df.iterrows():
    lat = row['Y좌표']
    lng = row['X좌표']
    # 행정동 정보를 가져오기 위해 region_type 매개변수 추가하여 URL 업데이트
    response_text = json_request(f"{URL}?x={lng}&y={lat}&input_coord=WGS84&output_coord=TM&region_type=H")
    response_json = json.loads(response_text)

    if 'documents' in response_json:
        # [0]은 법정동 데이터프레임
        # [1]은 행정동 데이터프레임 만들 수 있음
        document = response_json['documents'][1]

        region_type_list.append(document.get('region_type', ''))
        code_list.append(document.get('code', ''))
        address_name_list.append(document.get('address_name', ''))
        region_1depth_name_list.append(document.get('region_1depth_name', ''))
        region_2depth_name_list.append(document.get('region_2depth_name', ''))
        region_3depth_name_list.append(document.get('region_3depth_name', ''))
        region_4depth_name_list.append(document.get('region_4depth_name', ''))
        x_list.append(document.get('x', ''))
        y_list.append(document.get('y', ''))

# 결과 데이터프레임 생성
bs_result_df = pd.DataFrame({
    'region_type': region_type_list,
    'code': code_list,
    'address_name': address_name_list,
    'region_1depth_name': region_1depth_name_list,
    'region_2depth_name': region_2depth_name_list,
    'region_3depth_name': region_3depth_name_list,
    'region_4depth_name': region_4depth_name_list,
    'x': x_list,
    'y': y_list
})


2024-03-26 04:13:19.455392 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9877523&y=37.56980554&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:13:19.971867 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9965206&y=37.57943293&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:13:20.365025 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9982508&y=37.58258001&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:13:20.826092 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9876131&y=37.56857927&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:13:21.288462 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.001744&y=37.586243&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:13:21.711128 : success for request [https://dapi.kakao.com/v2/lo

2024-03-26 04:13:41.101155 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9764072&y=37.57004483&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:13:41.503157 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.971248&y=37.570023&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:13:41.984134 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9728386&y=37.57033689&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:13:42.419383 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.975837&y=37.570365&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:13:42.948037 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9769976&y=37.57364893&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:13:43.393133 : success for request [https://dapi.kakao.com/v2/local

2024-03-26 04:14:03.000225 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9614303&y=37.57134302&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:03.420861 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9885394&y=37.57754057&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:03.942258 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9800948&y=37.5692639&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:04.394421 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9849267&y=37.56868439&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:04.852209 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9908172&y=37.56844532&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:05.348305 : success for request [https://dapi.kakao.com/v2/

2024-03-26 04:14:23.452432 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9721318&y=37.56990034&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:23.957792 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9937794&y=37.56868879&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:24.516625 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0080377&y=37.56993039&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:25.072738 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0130967&y=37.57004196&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:25.592283 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0181256&y=37.57089499&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:26.076535 : success for request [https://dapi.kakao.com/v2

2024-03-26 04:14:45.859260 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9622629&y=37.60706212&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:46.305534 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9640686&y=37.56878711&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:46.881409 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0127581&y=37.5812648&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:47.414430 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9859477&y=37.58759481&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:47.980980 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.9878088&y=37.58308055&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 04:14:48.423742 : success for request [https://dapi.kakao.com/v2/

...

2024-03-26 05:47:13.533453 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.174408&y=37.568207&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:47:14.033916 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1657709&y=37.55555768&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:47:14.560261 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1801379&y=37.56169242&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:47:15.024496 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1799003&y=37.56171621&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:47:15.659570 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.174627&y=37.564251&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:47:16.234823 : success for request [https://dapi.kakao.com/v2/local

2024-03-26 05:47:37.233166 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1816669&y=37.55125906&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:47:37.685424 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1817343&y=37.55137165&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:47:38.200280 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1773711&y=37.55792621&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:47:38.758625 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.168922&y=37.57653738&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:47:39.220649 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.146753&y=37.54008805&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:47:39.705598 : success for request [https://dapi.kakao.com/v2/l

2024-03-26 05:48:00.014799 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.126183&y=37.53165&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:00.507196 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1353709&y=37.54808355&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:01.185303 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1280545&y=37.53105829&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:01.632791 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1312629&y=37.53014556&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:02.035484 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.131286&y=37.54266537&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:02.536338 : success for request [https://dapi.kakao.com/v2/loca

2024-03-26 05:48:24.029620 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1317313&y=37.54297665&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:24.527199 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1376866&y=37.550002&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:25.094231 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1426911&y=37.55064585&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:25.546484 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1429223&y=37.55070224&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:26.082329 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1353379&y=37.54807047&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:26.574954 : success for request [https://dapi.kakao.com/v2/l

2024-03-26 05:48:47.160211 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1430234&y=37.5454039&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:47.648835 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1427437&y=37.54558894&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:48.106586 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1441907&y=37.55414459&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:48.540579 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1443599&y=37.5533716&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:48.956521 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1285487&y=37.55331842&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 05:48:49.500176 : success for request [https://dapi.kakao.com/v2/l

In [55]:
bs_result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11261 entries, 0 to 11260
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   region_type         11261 non-null  object 
 1   code                11261 non-null  object 
 2   address_name        11261 non-null  object 
 3   region_1depth_name  11261 non-null  object 
 4   region_2depth_name  11261 non-null  object 
 5   region_3depth_name  11261 non-null  object 
 6   region_4depth_name  11261 non-null  object 
 7   x                   11261 non-null  float64
 8   y                   11261 non-null  float64
dtypes: float64(2), object(7)
memory usage: 791.9+ KB


In [56]:
bs_result_df.to_csv('bus_stop_df_counts.csv', index=False)

In [58]:
bs_result_df = bs_result_df.loc[bs_result_df['region_2depth_name']=='강서구']
bs_result_df = bs_result_df.groupby('region_3depth_name').size().reset_index(name='버스 정류장 개수')

In [4]:
ansim_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   자치구      3168 non-null   object 
 1   안심 주소    3168 non-null   object 
 2   위도       3168 non-null   float64
 3   경도       3168 non-null   float64
 4   CCTV 수량  3168 non-null   int64  
 5   수정 일시    3168 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 148.6+ KB


In [5]:
ansim_df.head()

,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
0,강서구,(공사중)C231003(A1069)_염창동 280-21,37.5486,126.8712,1,2022-12-01
1,강서구,(교체예정) C181022(SC121) - 마곡중앙5로 마곡중학교 맞은편,37.5670,126.8219,1,2022-12-01
2,강서구,(교체예정) S312010_화곡본동 748-24 화일초등학교 병설유치원,37.5405,126.8480,1,2022-12-01
3,강서구,(교체예정)C181011(SC131) - 마곡중앙로 국제업무단지 CP2앞(보조1),37.5634,126.8271,1,2022-12-01
4,강서구,(교체예정)C181026(SC106) - 마곡서1로 8단지 공원부지앞 삼거리,37.5651,126.8193,1,2022-12-01


In [7]:
## 카카오 API를 활용한 역 지오코딩(행정동 기준)

import pandas as pd
import json
import requests
from datetime import datetime

APP_KEY = "d5a2db9d678853381b40404881ef63c3"
URL = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"

def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': f'KakaoAK {APP_KEY}'}
    resp = requests.get(url, headers=headers)
    print('%s : success for request [%s]' % (datetime.now(), url))

    return resp.text

region_type_list = []
code_list = []
address_name_list = []
region_1depth_name_list = []
region_2depth_name_list = []
region_3depth_name_list = []
region_4depth_name_list = []
x_list = []
y_list = []

# 데이터프레임에 있는 각 좌표에 대해 요청 실행
for index, row in ansim_df.iterrows():
    lat = row['위도']
    lng = row['경도']
    # 행정동 정보를 가져오기 위해 region_type 매개변수 추가하여 URL 업데이트
    response_text = json_request(f"{URL}?x={lng}&y={lat}&input_coord=WGS84&output_coord=TM&region_type=H")
    response_json = json.loads(response_text)

    if 'documents' in response_json:
        # [0]은 법정동 데이터프레임
        # [1]은 행정동 데이터프레임 만들 수 있음
        document = response_json['documents'][1]

        region_type_list.append(document.get('region_type', ''))
        code_list.append(document.get('code', ''))
        address_name_list.append(document.get('address_name', ''))
        region_1depth_name_list.append(document.get('region_1depth_name', ''))
        region_2depth_name_list.append(document.get('region_2depth_name', ''))
        region_3depth_name_list.append(document.get('region_3depth_name', ''))
        region_4depth_name_list.append(document.get('region_4depth_name', ''))
        x_list.append(document.get('x', ''))
        y_list.append(document.get('y', ''))

# 결과 데이터프레임 생성
as_result_df = pd.DataFrame({
    'region_type': region_type_list,
    'code': code_list,
    'address_name': address_name_list,
    'region_1depth_name': region_1depth_name_list,
    'region_2depth_name': region_2depth_name_list,
    'region_3depth_name': region_3depth_name_list,
    'region_4depth_name': region_4depth_name_list,
    'x': x_list,
    'y': y_list
})

2024-03-26 18:23:53.504484 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8712&y=37.5486&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:23:54.537893 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8219&y=37.567&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:23:55.641831 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.848&y=37.5405&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:23:56.745981 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8271&y=37.5634&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:23:57.907939 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8193&y=37.5651&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:23:59.082846 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=12

2024-03-26 18:24:51.136120 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.817&y=37.5546&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:24:52.303326 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8148&y=37.5576&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:24:53.479562 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8491&y=37.5537&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:24:54.557315 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8598&y=37.5547&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:24:55.622189 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8608&y=37.5508&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:24:56.694116 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=1

2024-03-26 18:25:42.783921 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.843&y=37.5346&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:25:43.848335 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8514&y=37.5529&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:25:44.928050 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8178&y=37.5718&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:25:45.995314 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8449&y=37.5414&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:25:47.046412 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8318&y=37.5515&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:25:48.126970 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=1

2024-03-26 18:26:33.900221 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8583&y=37.5625&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:26:34.959059 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8583&y=37.5625&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:26:36.086033 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8583&y=37.5625&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:26:37.170220 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8616&y=37.5592&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:26:38.324805 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8616&y=37.5592&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:26:39.396673 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=

2024-03-26 18:27:25.122176 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8108&y=37.5605&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:27:26.193403 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8108&y=37.5605&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:27:27.320398 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8108&y=37.5605&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:27:28.451280 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8092&y=37.5599&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:27:29.381980 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8093&y=37.5599&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 18:27:30.440151 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=

...

2024-03-26 19:15:54.352655 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8147&y=37.5783&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:15:54.869071 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8147&y=37.5783&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:15:55.375296 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8194&y=37.5774&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:15:55.907692 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8194&y=37.5774&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:15:56.570443 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8681&y=37.5523&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:15:57.062560 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=

2024-03-26 19:16:21.011633 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8384&y=37.5404&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:16:21.531515 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8384&y=37.5404&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:16:22.119730 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8359&y=37.5349&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:16:22.578390 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8378&y=37.5317&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:16:23.036762 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8445&y=37.5322&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:16:23.511456 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=

2024-03-26 19:16:46.429456 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8488&y=37.5499&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:16:46.898863 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8488&y=37.5499&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:16:47.443253 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8536&y=37.5512&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:16:47.979573 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8536&y=37.5512&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:16:48.489566 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8536&y=37.5512&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:16:49.040224 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=

2024-03-26 19:17:12.783852 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8383&y=37.5377&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:17:13.311167 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8448&y=37.5291&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:17:13.827955 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8424&y=37.5297&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:17:14.324378 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8547&y=37.5364&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:17:14.856842 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8566&y=37.5303&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:17:15.423706 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=

2024-03-26 19:17:39.073337 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8614&y=37.5535&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:17:39.636285 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8613&y=37.553&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:17:40.164433 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8389&y=37.5394&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:17:40.706687 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8298&y=37.5579&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:17:41.154408 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8298&y=37.5579&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 19:17:41.684379 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=1

In [8]:
as_df = as_result_df.loc[as_result_df['region_2depth_name']=='강서구'].groupby('region_3depth_name').size().reset_index(name='안심cctv개수')

In [11]:
as_df.to_csv('as_df.csv',index=False)

## 열 인덱스명 통일 후 merge

In [64]:
#as_df.rename(columns={'region_3depth_name': '행정동'}, inplace=True)
bs_result_df.rename(columns={'region_3depth_name': '행정동'}, inplace=True)
cw_result_df.rename(columns={'region_3depth_name': '행정동'}, inplace=True)
merged_df_1 = pd.merge(bs_result_df, cw_result_df, on='행정동')

print(merged_df_1)

     행정동  버스 정류장 개수  횡단보도 수
0   가양1동         58     127
1   가양2동         12      18
2   가양3동         17      22
3    공항동         60      60
4   등촌1동         15      17
5   등촌2동          9       3
6   등촌3동         47      45
7   발산1동         65     112
8   방화1동         43      56
9   방화2동         35      35
10  방화3동         20      50
11   염창동         36      14
12  우장산동         27      19
13  화곡1동         27      33
14  화곡2동         28       4
15  화곡3동          9       7
16  화곡4동         31       5
17  화곡6동         29      11
18  화곡8동         26       6
19  화곡본동         25       6


In [65]:
merged_df_1.to_csv('merged_df_1.csv', index=False)

In [ ]:
merged_df.to_csv('merged_df.csv',index=False)

In [4]:
df = pd.read_csv('C:/Users/PC/Jupyter_file/강서구_빅데이터공모전/data/12_22_death.csv',encoding='cp949')

In [5]:
df.head()

,발생년,발생년월일시,주야,요일,사망자수,사상자수,중상자수,경상자수,부상신고자수,발생지시도,...,사고유형,법규위반,도로형태_대분류,도로형태,당사자종별_1당_대분류,당사자종별_2당_대분류,발생위치X_UTMK,발생위치Y_UTMK,경도,위도
0,2012,2012010101,야간,일,1,1,0,0,0,서울,...,차도통행중,안전운전 의무 불이행,단일로,기타단일로,승용차,보행자,949860,1957179,126.931890,37.612680
1,2012,2012010101,야간,일,1,6,5,0,0,전북,...,정면충돌,중앙선 침범,단일로,기타단일로,승용차,승용차,946537,1737695,126.909523,35.633956
2,2012,2012010108,주간,일,1,1,0,0,0,충남,...,공작물충돌,안전운전 의무 불이행,단일로,기타단일로,승용차,없음,940016,1832833,126.830281,36.491268
3,2012,2012010110,주간,일,2,2,0,0,0,경남,...,측면충돌,과속,교차로,교차로내,승합차,승용차,1059321,1748774,128.155984,35.733503
4,2012,2012010103,야간,일,1,1,0,0,0,경북,...,도로이탈 추락,안전운전 의무 불이행,단일로,기타단일로,승용차,없음,1070222,1834630,128.284180,36.506769


In [26]:
df = df.loc[df['발생지시군구']=='강서구'].loc[df['발생지시도']=='서울']

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 34241 to 42157
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   발생년           45 non-null     int64  
 1   발생년월일시        45 non-null     int64  
 2   주야            45 non-null     object 
 3   요일            45 non-null     object 
 4   사망자수          45 non-null     int64  
 5   사상자수          45 non-null     int64  
 6   중상자수          45 non-null     int64  
 7   경상자수          45 non-null     int64  
 8   부상신고자수        45 non-null     int64  
 9   발생지시도         45 non-null     object 
 10  발생지시군구        45 non-null     object 
 11  사고유형_대분류      45 non-null     object 
 12  사고유형_중분류      45 non-null     object 
 13  사고유형          45 non-null     object 
 14  법규위반          45 non-null     object 
 15  도로형태_대분류      45 non-null     object 
 16  도로형태          45 non-null     object 
 17  당사자종별_1당_대분류  45 non-null     object 
 18  당사자종별_2당_대분류  45 non-null     

In [36]:
df = df.loc[(df['발생년'] >= 2020)]

In [38]:
## 카카오 API를 활용한 역 지오코딩(행정동 기준)

import pandas as pd
import json
import requests
from datetime import datetime

APP_KEY = "d5a2db9d678853381b40404881ef63c3"
URL = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"

def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': f'KakaoAK {APP_KEY}'}
    resp = requests.get(url, headers=headers)
    print('%s : success for request [%s]' % (datetime.now(), url))

    return resp.text

region_type_list = []
code_list = []
address_name_list = []
region_1depth_name_list = []
region_2depth_name_list = []
region_3depth_name_list = []
region_4depth_name_list = []
x_list = []
y_list = []

# 데이터프레임에 있는 각 좌표에 대해 요청 실행
for index, row in df.iterrows():
    lat = row['위도']
    lng = row['경도']
    # 행정동 정보를 가져오기 위해 region_type 매개변수 추가하여 URL 업데이트
    response_text = json_request(f"{URL}?x={lng}&y={lat}&input_coord=WGS84&output_coord=TM&region_type=H")
    response_json = json.loads(response_text)

    if 'documents' in response_json:
        # [0]은 법정동 데이터프레임
        # [1]은 행정동 데이터프레임 만들 수 있음
        document = response_json['documents'][1]

        region_type_list.append(document.get('region_type', ''))
        code_list.append(document.get('code', ''))
        address_name_list.append(document.get('address_name', ''))
        region_1depth_name_list.append(document.get('region_1depth_name', ''))
        region_2depth_name_list.append(document.get('region_2depth_name', ''))
        region_3depth_name_list.append(document.get('region_3depth_name', ''))
        region_4depth_name_list.append(document.get('region_4depth_name', ''))
        x_list.append(document.get('x', ''))
        y_list.append(document.get('y', ''))

# 결과 데이터프레임 생성
death_result_df = pd.DataFrame({
    'region_type': region_type_list,
    'code': code_list,
    'address_name': address_name_list,
    'region_1depth_name': region_1depth_name_list,
    'region_2depth_name': region_2depth_name_list,
    'region_3depth_name': region_3depth_name_list,
    'region_4depth_name': region_4depth_name_list,
    'x': x_list,
    'y': y_list
})

2024-03-26 20:00:21.408449 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.81691579&y=37.57337907&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 20:00:21.996459 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.85315982&y=37.56116184&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 20:00:22.462309 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.87554577&y=37.54703392&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 20:00:22.947573 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.8336788&y=37.55906694&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 20:00:23.437508 : success for request [https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.81904781&y=37.55040539&input_coord=WGS84&output_coord=TM&region_type=H]
2024-03-26 20:00:23.899472 : success for request [https://dapi.kakao.co

In [39]:
death_df = death_result_df.loc[death_result_df['region_2depth_name']=='강서구'].groupby('region_3depth_name').size().reset_index(name='20~22사망자 수')

In [41]:
death_df.head(15)

,region_3depth_name,20~22사망자 수
0,가양1동,3
1,가양2동,2
2,공항동,3
3,등촌1동,3
4,등촌3동,3
5,발산1동,3
6,방화1동,1
7,방화2동,6
8,방화3동,3
9,염창동,9


In [42]:
death_df.to_csv("death_df.csv",index=False)

In [6]:
import pandas as pd
final_df = pd.read_csv("C:/Users/PC/Downloads/승승장구 - 로길 시트.csv",encoding = 'cp949')

In [7]:
final_df.head()

,행정동,옐로카펫,"노인,장애인보호구역",어린이집,유치원,초등학교,중학교,고등학교,특수학교,평생학교,...,불법주정차_단속건수,노인_인구,노인_사고건수,어린이_인구,어린이_사고건수,2020_2022_교통사고_사망건수,2020_2022_사고건수,안심cctv개수,버스 정류장 개수,횡단보도 수
0,염창동,2,0,29,1,3,2,0,0,0,...,7495,"4,872",2,"7,791",1,9,265,124,36,14
1,등촌1동,0,1,12,1,1,0,0,0,0,...,6341,"3,226",6,"2,375",2,4,133,80,15,17
2,등촌2동,1,0,11,2,2,2,2,0,0,...,1888,"3,520",2,"2,372",2,0,81,109,9,3
3,등촌3동,4,2,16,5,5,3,4,0,0,...,2824,"7,442",6,"3,047",0,2,217,93,47,45
4,화곡본동,2,2,23,4,2,0,0,0,2,...,4098,"5,379",4,"3,596",0,1,238,320,25,6


In [8]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   행정동                         20 non-null     object
 1   옐로카펫                        20 non-null     int64 
 2   노인,장애인보호구역                  20 non-null     int64 
 3   어린이집                        20 non-null     int64 
 4   유치원                         20 non-null     int64 
 5   초등학교                        20 non-null     int64 
 6   중학교                         20 non-null     int64 
 7   고등학교                        20 non-null     int64 
 8   특수학교                        20 non-null     int64 
 9   평생학교                        20 non-null     int64 
 10  학교(전체)                      20 non-null     int64 
 11  단란주점                        20 non-null     int64 
 12  유흥주점                        20 non-null     int64 
 13  공동주택                        20 non-null     int64 
 